importing required libraries

In [134]:
import json
import os
import time
import hashlib

Constants

In [135]:
VERSION = '1'
PREV_BLOCK_HASH = '0000111100000000000000000000000000000000000000000000000000000000'
DIFFICULTY_TARGET = '0000ffff00000000000000000000000000000000000000000000000000000000'
COINBASE_REWARD = 50
COINBASE_TXID = 'unique_coinbase_txid'
total_fees = 0

Helper function to calculate the Merkle root

In [136]:
def merkle_root(txids): 
    # If the list of txids is empty, return a dummy value
    if not txids: 
        return '0'*64
    # If there is only one txid, return it as the root
    if len(txids) == 1: 
        return txids[0]
    # If the number of txids is odd, duplicate the last one
    if len(txids) % 2 == 1: 
        txids.append(txids[-1])
    # Recursively calculate the next level of the tree
    new_txids = []
    for i in range(0, len(txids), 2): 
        new_txids.append(hashlib.sha256((txids[i] + txids[i+1]).encode()).hexdigest())
    return merkle_root(new_txids)

Function to validate a transaction

In [137]:
def validate_transaction(transaction): 
    # Extract the integer value if 'prevout' or 'vout' is a dictionary containing an integer value
    prevout_value = sum([prevout.get('value', 0) for prevout in transaction.get('prevout', [])])
    vout_value = sum([vout.get('value', 0) for vout in transaction.get('vout', [])])
    txid = transaction.get('txid', '')
    
    # Calculate the transaction fee
    fee = prevout_value - vout_value
    
    # Check if the transaction is valid (prevout > vout) and txid is not empty
    is_valid = prevout_value > vout_value and bool(txid)
    
    # Return both the fee and whether the transaction is valid
    return fee, is_valid

Function to mine a block

In [138]:
def mine_block(block_header): 
    nonce = 0
    while True: 
        block_header['nonce'] = nonce
        block_hash = hashlib.sha256(json.dumps(block_header).encode()).hexdigest()
        if int(block_hash, 16) < int(DIFFICULTY_TARGET, 16): 
            return nonce, block_hash
        nonce += 1

Main function to read, validate, mine, and output block details

In [139]:
def main(): 
    transactions = []
    total_fees = 0
    for filename in os.listdir('mempool'): 
        if filename.endswith('.json'): 
            with open(os.path.join('mempool', filename), 'r') as file: 
                transaction = json.load(file)
                fee, is_valid = validate_transaction(transaction)
                if is_valid: 
                    transactions.append(transaction.get('txid', ''))
                    total_fees += fee

Define the coinbase transaction

In [140]:
coinbase_transaction = {
        'txid':  'unique_coinbase_txid',
        'vin':  [{'coinbase':  'data'}],
        'vout':  [{'value':  COINBASE_REWARD + total_fees}]
    }

Insert the coinbase transaction at the beginning of the transactions list

In [141]:
transactions.insert(0, coinbase_transaction['txid'])

Construct the block

In [142]:
block_header = {
        'version':  VERSION,
        'previous_block_hash':  PREV_BLOCK_HASH,
        'merkle_root':  merkle_root(transactions),
        'timestamp':  int(time.time()),
        'difficulty_target':  DIFFICULTY_TARGET,
        'nonce':  0  # Placeholder for nonce
    }

Mine the block

In [143]:
nonce, block_hash = mine_block(block_header)

serialise the block header

In [144]:
block_header_serialized = json.dumps({
        "version":  VERSION,
        "previous_block_hash":  PREV_BLOCK_HASH,
        "merkle_root":  merkle_root(transactions),
        "timestamp":  int(time.time()),
        "difficulty_target":  DIFFICULTY_TARGET,
        "nonce":  nonce
    }, indent=2)

serialise the coinbase transaction

In [145]:
coinbase_transaction_serialized = json.dumps({
        "txid":  COINBASE_TXID,
        "vin":  [{
            "coinbase":  "04ffff001d0104455468652054696d65732030332f4a616e2f32303233204368616e63656c6c6f72206f6e20626974636f696e2062756c6c",
            "sequence":  4294967295
        }],
        "vout":  [{
            "value":  COINBASE_REWARD,
            "scriptPubKey":  "4104678afdb0fe5548271967f1a67130b7105cd6a828e03909a67962e0ea1f61deb649f6bc3f4cef38c4f35504e51ec112de5c384df7ba0b8d578a4c702b6bf11d5fac"
        }]
    }, indent=2)

Output the block details

In [146]:
with open('output.txt', 'w') as file: 
        file.write("Block Header: \n")
        file.write(block_header_serialized + "\n\n")
        file.write("Serialized Coinbase Transaction: \n")
        file.write(coinbase_transaction_serialized + "\n\n")
        file.write("Transaction IDs: \n")
        for txid in transactions: 
            file.write(txid + "\n")

print("Block mined and output.txt created successfully.")

Block mined and output.txt created successfully.


Create run.sh

In [147]:
with open('run.sh', 'w') as file: 
        file.write('#!/bin/bash\n')
        file.write('python main.py\n')

print("Block mined and output.txt created successfully.")

Block mined and output.txt created successfully.


In [148]:
if __name__ == '__main__': 
    main()